In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
flights_sample_small=pd.read_csv('flights_sample_small.csv')

## 1.   knock-on Feature

In [3]:
X=flights_sample_small[['fl_date','tail_num','crs_dep_time','crs_arr_time']]

In [4]:
X['crs_dep_time'] = X['crs_dep_time'].astype(str).str.zfill(4)
X['crs_arr_time'] = X['crs_arr_time'].astype(str).str.zfill(4)

def get_hour_min(x):    
    return str(int(x[:2]))+':'+str(int(x[-2:]))

X['crs_dep_time'] = X['crs_dep_time'].apply(lambda x: get_hour_min(x))
X['crs_arr_time'] = X['crs_arr_time'].apply(lambda x: get_hour_min(x))

#adding zero so that each time value consists of 4 digits
X['crs_dep_time'] = X['crs_dep_time'].str.replace('24:0', '00:00')
X['crs_arr_time'] = X['crs_arr_time'].str.replace('24:0', '00:00')


# convert time columns to datetime
X['crs_dep_time'] = pd.to_datetime(X['crs_dep_time'], format='%H:%M')
X['crs_arr_time'] = pd.to_datetime(X['crs_arr_time'], format='%H:%M')


In [5]:
X['knock-on']=np.nan
flights_sample_small_clean=X
flights_sample_small_clean

,fl_date,tail_num,crs_dep_time,crs_arr_time,knock-on
0,2019-01-01,N576NN,1900-01-01 06:20:00,1900-01-01 09:56:00,NaN
1,2019-01-01,N575NN,1900-01-01 21:57:00,1900-01-01 23:11:00,NaN
2,2019-01-01,N254PS,1900-01-01 11:51:00,1900-01-01 13:25:00,NaN
3,2019-01-01,N254PS,1900-01-01 10:09:00,1900-01-01 11:26:00,NaN
4,2019-01-01,N710PS,1900-01-01 20:00:00,1900-01-01 21:34:00,NaN
...,...,...,...,...,...
19517,2019-01-01,N909DA,1900-01-01 19:03:00,1900-01-01 20:33:00,NaN
19518,2019-01-01,N344NW,1900-01-01 16:56:00,1900-01-01 18:57:00,NaN
19519,2019-01-01,N659DL,1900-01-01 09:32:00,1900-01-01 15:32:00,NaN
19520,2019-01-01,N365DN,1900-01-01 08:30:00,1900-01-01 10:47:00,NaN


In [6]:
#unique tail nums and dates
tail_num_val=list(X['tail_num'].unique())
fl_date_val=list(X['fl_date'].unique())

In [33]:
#define a function to filter the table
#This filtering only provides information in a single day for a specific airplane
def filtered_sorted(tail_num_val,fl_date_val):
    a=X[(X['tail_num'] == tail_num_val) & (X['fl_date'] == fl_date_val)].sort_values('crs_dep_time').reset_index()
    return a

In [8]:
for i in tail_num_val:
    #print (i)
    for j in fl_date_val:
        #print (j)
        df=filtered_sorted(i,j)
        #print (df)
        for k in range(len(df)-1):
            a=df.loc[k+1, 'index'] #index_X
            X.loc[a,'knock-on'] #value of knock-on
            c=(df.loc[k+1, 'crs_dep_time'])
            d=(df.loc[k, 'crs_arr_time'])
            X.loc[a,'knock-on']=c-d
            X.loc[a,'knock-on']=X.loc[a,'knock-on']

In [41]:
#extract only the time part
X['crs_dep_time']=X['crs_dep_time'].dt.time
X['crs_arr_time']=X['crs_arr_time'].dt.time

AttributeError: Can only use .dt accessor with datetimelike values

In [10]:
X

,fl_date,tail_num,crs_dep_time,crs_arr_time,knock-on
0,2019-01-01,N576NN,06:20:00,09:56:00,NaN
1,2019-01-01,N575NN,21:57:00,23:11:00,0 days 00:30:00
2,2019-01-01,N254PS,11:51:00,13:25:00,0 days 00:25:00
3,2019-01-01,N254PS,10:09:00,11:26:00,0 days 01:01:00
4,2019-01-01,N710PS,20:00:00,21:34:00,0 days 00:40:00
...,...,...,...,...,...
19517,2019-01-01,N909DA,19:03:00,20:33:00,0 days 00:50:00
19518,2019-01-01,N344NW,16:56:00,18:57:00,0 days 00:46:00
19519,2019-01-01,N659DL,09:32:00,15:32:00,NaN
19520,2019-01-01,N365DN,08:30:00,10:47:00,NaN


In [19]:
# transform knock on feature to only extract the time

Y=X.copy()
Y['knock-on'] = Y.apply(lambda r: np.nan if type(r['knock-on']) == float else r['knock-on'], axis=1)

Y['knock-on'] = (pd.Timestamp('now').normalize() + Y['knock-on']).dt.time
Y

,fl_date,tail_num,crs_dep_time,crs_arr_time,knock-on
0,2019-01-01,N576NN,06:20:00,09:56:00,NaT
1,2019-01-01,N575NN,21:57:00,23:11:00,00:30:00
2,2019-01-01,N254PS,11:51:00,13:25:00,00:25:00
3,2019-01-01,N254PS,10:09:00,11:26:00,01:01:00
4,2019-01-01,N710PS,20:00:00,21:34:00,00:40:00
...,...,...,...,...,...
19517,2019-01-01,N909DA,19:03:00,20:33:00,00:50:00
19518,2019-01-01,N344NW,16:56:00,18:57:00,00:46:00
19519,2019-01-01,N659DL,09:32:00,15:32:00,NaT
19520,2019-01-01,N365DN,08:30:00,10:47:00,NaT


In [21]:
Y.to_csv('./tables/flights_sample_small_knock_on.csv', index=False)

##  Origin and Departure plane traffic

In [59]:
df[df['crs_dep_time'] == '21:57:00']

,fl_date,tail_num,crs_dep_time,crs_arr_time,knock-on,origin_airport_id,dest_airport_id
1,2019-01-01,N575NN,21:57:00,23:11:00,00:30:00,13577,11057
2676,2019-01-01,N701UW,21:57:00,23:24:00,00:45:00,14524,11057
13180,2019-01-01,N806JB,21:57:00,06:22:00,01:20:00,14893,10721


In [137]:
Z=Y.copy()

#Convert dates to string type
Z['crs_dep_time']=Z['crs_dep_time'].apply(lambda x: x.strftime('%H:%M:%S'))
Z['crs_arr_time']=Z['crs_arr_time'].apply(lambda x: x.strftime('%H:%M:%S'))

# adding columns to the the previous table
Z['origin_airport_id']=flights_sample_small['origin_airport_id']
Z['dest_airport_id']=flights_sample_small['dest_airport_id']

#adding feature columns
Z['origin_plane_traffic']=np.nan
Z['dest_plane_traffic']=np.nan

df=Z.copy()
len(Z)

19522

In [130]:
#define a function to get origin and dest plane traffic 30 min before and after departure/arrival

def origin_dest_plane_traffic(fl_date, tail_num, crs_dep_time, crs_arr_time):
    index = df[(df['fl_date'] == fl_date) & (df['tail_num'] == tail_num) &  (df['crs_dep_time'] == crs_dep_time)].index[0]
    origin_airport_id=df.loc[index,'origin_airport_id']
    time_diff = pd.Timedelta(minutes=30)
         
    #plane traffic in departure
    
    #filter the table based on origin airport id and flight date
    filtered_sorted = df[(df['fl_date'] == fl_date) & (df['origin_airport_id'] == origin_airport_id)]
    #covert string to datetime
    filtered_sorted['crs_dep_time'] = pd.to_datetime(filtered_sorted['crs_dep_time'])    
    time_value = pd.to_datetime(crs_dep_time)
    
    #filter further to include the time 30 min before and after departure
    origin=len(filtered_sorted[(filtered_sorted['crs_dep_time'] >= time_value - time_diff) & (filtered_sorted['crs_dep_time'] <= time_value + time_diff)])
    
    #plane traffic in arrival
    dest_airport_id=df.loc[index,'dest_airport_id'] 
    
    filtered_sorted_2 = df[(df['fl_date'] == fl_date) & (df['dest_airport_id'] == dest_airport_id)]
    filtered_sorted_2['crs_arr_time'] = pd.to_datetime(filtered_sorted_2['crs_arr_time'])
    
    #covert string to datetime
    time_value = pd.to_datetime(crs_arr_time)   
    
    dept=len(filtered_sorted_2[(filtered_sorted_2['crs_arr_time'] >= time_value - time_diff) & (filtered_sorted_2['crs_arr_time'] <= time_value + time_diff)])
    
    
    #return a list for origin and arrival plane traffic
    return [origin,dept]

#sample for testing
c='06:20:00'
d='09:56:00'
origin_dest_plane_traffic('2019-01-01','N576NN',c,d)


[6, 36]

In [135]:
for i in range(len(df)):
    a=df.loc[i,'fl_date']
    b=df.loc[i,'tail_num']
    c=df.loc[i,'crs_dep_time'] 
    d=df.loc[i,'crs_arr_time']
    e=origin_dest_plane_traffic(a,b,c,d)
   
    df.loc[i,'origin_plane_traffic']=e[0]    
    df.loc[i,'dest_plane_traffic']=e[1]
   

df.head(20)

,fl_date,tail_num,crs_dep_time,crs_arr_time,knock-on,origin_airport_id,dest_airport_id,origin_plane_traffic,dest_plane_traffic
0,2019-01-01,N576NN,06:20:00,09:56:00,NaT,11423,11057,6.0,36.0
1,2019-01-01,N575NN,21:57:00,23:11:00,00:30:00,13577,11057,2.0,63.0
2,2019-01-01,N254PS,11:51:00,13:25:00,00:25:00,10529,11278,3.0,17.0
3,2019-01-01,N254PS,10:09:00,11:26:00,01:01:00,11278,10529,18.0,3.0
4,2019-01-01,N710PS,20:00:00,21:34:00,00:40:00,11278,11996,26.0,2.0
5,2019-01-01,N515AE,12:15:00,13:47:00,NaT,13931,11057,7.0,55.0
6,2019-01-01,N242JS,22:20:00,23:43:00,01:19:00,11278,11042,13.0,8.0
7,2019-01-01,N703PS,19:05:00,20:04:00,00:30:00,10431,11057,1.0,13.0
8,2019-01-01,N703PS,17:45:00,18:35:00,00:41:00,11057,10431,56.0,1.0
9,2019-01-01,N581NN,14:43:00,15:55:00,00:44:00,11057,13342,60.0,6.0


In [138]:
df.to_csv('./tables/flights_sample_small_knock_on_traffic.csv', index=False)